In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas_read_xml as pdx
from pandas_read_xml import auto_separate_tables

In [3]:
df_2 = pdx.read_xml('export.xml', transpose=True)

In [4]:
df_2

,HealthData
@locale,en_US
ExportDate,{'@value': '2020-08-18 21:01:15 -0500'}
Me,{'@HKCharacteristicTypeIdentifierDateOfBirth':...
Record,"[{'@type': 'HKQuantityTypeIdentifierHeight', '..."


In [14]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, @locale to Record
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   HealthData  4 non-null      object
dtypes: object(1)
memory usage: 64.0+ bytes


In [5]:
def list_separate_tables(df: pd.DataFrame) -> list:
    potential_separate_table = list(set([
        '|'.join(column.split('|')[:-1])
        for column in df.columns
        if determine_flatten_action_for_column(df, column).__name__ != "do_nothing"
    ]))
    potential_separate_table.sort()

    separate_tables_list = potential_separate_table
    for i, potential in enumerate(potential_separate_table):
        if (i < len(potential_separate_table) - 1) and (potential_separate_table[i+1].startswith(potential)):
            separate_tables_list.remove(potential_separate_table[i+1])
    if separate_tables_list == ['']:
        return ['main']+potential_separate_table
    else:
        return [column for column in df.columns
                if determine_flatten_action_for_column(df, column).__name__ != "do_nothing"]

In [7]:
list_separate_tables(df_2)

NameError: name 'determine_flatten_action_for_column' is not defined

In [15]:
data = df_2.pipe(auto_separate_tables, key_columns)

NameError: name 'key_columns' is not defined

In [16]:
#https://pypi.org/project/pandas-read-xml/

In [17]:




columns = ["name", "email", "age"]

datatframe = pd.DataFrame(columns = columns)

for node in root: 

    name = node.attrib.get("name")

    mail = node.find("email").text if node is not None else None

    age = node.find("age").text if node is not None else None

    datatframe = datatframe.append(pd.Series([name, mail, age], index = columns), ignore_index = True)

NameError: name 'root' is not defined

In [18]:
import xml.etree.ElementTree as etree

In [19]:
tree = etree.parse("export.xml")

In [20]:
root = tree.getroot()

In [21]:
root.tag, root.attrib

('HealthData', {'locale': 'en_US'})

In [63]:
root[5].attrib

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [60]:
df = pd.DataFrame(root[4].attrib, index=[0])

In [78]:
root[5].attrib['type']

'HKQuantityTypeIdentifierStepCount'

In [ ]:
## check to see if attribute is step counter
i = 4
while i < 67920:
    if root[i].attrib['type'] == 'HKQuantityTypeIdentifierStepCount':
    else:
        print(i)
        break
    i += 1
    

In [90]:
root[67919].attrib

{'type': 'HKQuantityTypeIdentifierDistanceWalkingRunning',
 'sourceName': "Noah's iPhone",
 'sourceVersion': '9.3.1',
 'device': '<<HKDevice: 0x282513d90>, name:iPhone, manufacturer:Apple, model:iPhone, hardware:iPhone8,1, software:9.3.1>',
 'unit': 'mi',
 'creationDate': '2016-06-03 17:38:46 -0500',
 'startDate': '2016-06-03 16:40:09 -0500',
 'endDate': '2016-06-03 16:40:52 -0500',
 'value': '0.0351199'}

In [ ]:
i = 67919
while i < 1_000_000:
    if root[i].attrib['type'] == 'HKQuantityTypeIdentifierDistanceWalkingRunning':
    else:
        print(f'{i} is false and break')
        break
    i += 1

In [94]:
root[135827].attrib

{'type': 'HKQuantityTypeIdentifierFlightsClimbed',
 'sourceName': "Noah's iPhone",
 'sourceVersion': '9.3.1',
 'device': '<<HKDevice: 0x282592030>, name:iPhone, manufacturer:Apple, model:iPhone, hardware:iPhone8,1, software:9.3.1>',
 'unit': 'count',
 'creationDate': '2016-06-04 16:30:41 -0500',
 'startDate': '2016-06-04 15:45:08 -0500',
 'endDate': '2016-06-04 15:45:08 -0500',
 'value': '1'}

In [ ]:
i = 135827
while i < 1_000_000:
    if root[i].attrib['type'] == 'HKQuantityTypeIdentifierFlightsClimbed':
    else:
        print(f'{i} is false and break')
        break
    i += 1

In [97]:
root[137028].attrib

{'type': 'HKQuantityTypeIdentifierHeadphoneAudioExposure',
 'sourceName': "Noah's iPhone",
 'sourceVersion': '13.2.2',
 'device': '<<HKDevice: 0x282501b30>, name:AirPods, manufacturer:Apple Inc., model:0x200f, localIdentifier:4C:6B:E8:3A:54:08-tacl>',
 'unit': 'dBASPL',
 'creationDate': '2019-11-11 07:27:51 -0500',
 'startDate': '2019-11-11 07:16:51 -0500',
 'endDate': '2019-11-11 07:37:57 -0500',
 'value': '61.3887'}

In [ ]:
i = 4
type_list = []
while i < 1_000_000:
    if root[i].attrib['type'] in type_list:

    else:
        type_list.append(root[i].attrib['type'])
    i +=1
        

In [104]:
type_list

['HKQuantityTypeIdentifierStepCount',
 'HKQuantityTypeIdentifierDistanceWalkingRunning',
 'HKQuantityTypeIdentifierFlightsClimbed',
 'HKQuantityTypeIdentifierHeadphoneAudioExposure',
 'HKCategoryTypeIdentifierSleepAnalysis']

In [107]:
df = pd.DataFrame()
for i in range(4,137028):
    dict_new = root[i].attrib
    df = df.append(dict_new, ignore_index=True)


,creationDate,device,endDate,sourceName,sourceVersion,startDate,type,unit,value
0,2016-06-03 17:38:46 -0500,"<<HKDevice: 0x2825a8410>, name:iPhone, manufac...",2016-06-03 16:40:52 -0500,Noah's iPhone,9.3.1,2016-06-03 16:40:09 -0500,HKQuantityTypeIdentifierStepCount,count,70
1,2016-06-03 17:38:46 -0500,"<<HKDevice: 0x2825a83c0>, name:iPhone, manufac...",2016-06-03 16:58:26 -0500,Noah's iPhone,9.3.1,2016-06-03 16:57:23 -0500,HKQuantityTypeIdentifierStepCount,count,68
2,2016-06-03 18:39:21 -0500,"<<HKDevice: 0x2825a9270>, name:iPhone, manufac...",2016-06-03 17:35:45 -0500,Noah's iPhone,9.3.1,2016-06-03 17:34:29 -0500,HKQuantityTypeIdentifierStepCount,count,55
3,2016-06-03 18:39:21 -0500,"<<HKDevice: 0x2825aa030>, name:iPhone, manufac...",2016-06-03 17:48:03 -0500,Noah's iPhone,9.3.1,2016-06-03 17:43:01 -0500,HKQuantityTypeIdentifierStepCount,count,42
4,2016-06-03 18:39:21 -0500,"<<HKDevice: 0x2825a8cd0>, name:iPhone, manufac...",2016-06-03 17:49:54 -0500,Noah's iPhone,9.3.1,2016-06-03 17:48:03 -0500,HKQuantityTypeIdentifierStepCount,count,43
...,...,...,...,...,...,...,...,...,...
137019,2020-08-02 23:59:24 -0500,"<<HKDevice: 0x282501220>, name:iPhone, manufac...",2020-08-02 23:48:24 -0500,Noah's iPhone,13.5.1,2020-08-02 23:48:21 -0500,HKQuantityTypeIdentifierFlightsClimbed,count,1
137020,2020-08-03 08:46:39 -0500,"<<HKDevice: 0x2825033e0>, name:iPhone, manufac...",2020-08-03 08:35:06 -0500,Noah's iPhone,13.5.1,2020-08-03 08:35:03 -0500,HKQuantityTypeIdentifierFlightsClimbed,count,1
137021,2020-08-03 11:47:24 -0500,"<<HKDevice: 0x282503200>, name:iPhone, manufac...",2020-08-03 11:30:20 -0500,Noah's iPhone,13.5.1,2020-08-03 11:30:17 -0500,HKQuantityTypeIdentifierFlightsClimbed,count,1
137022,2020-08-15 09:36:32 -0500,"<<HKDevice: 0x282500780>, name:iPhone, manufac...",2020-08-15 09:33:55 -0500,Noah's iPhone,13.5.1,2020-08-15 09:24:02 -0500,HKQuantityTypeIdentifierFlightsClimbed,count,3


In [109]:
df.to_csv('xml.csv')

In [111]:
df.creationDate.value_counts().value_counts()

2      24891
1       4206
4       2978
6       1470
8       1042
       ...  
231        1
142        1
55         1
106        1
104        1
Name: creationDate, Length: 81, dtype: int64

In [112]:
df.startDate.value_counts().value_counts()

2    67087
1     2735
3       33
4        4
Name: startDate, dtype: int64

In [113]:
df.endDate.value_counts().value_counts()

2    66893
1     3114
3       24
4       13
Name: endDate, dtype: int64

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137024 entries, 0 to 137023
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   creationDate   137024 non-null  object
 1   device         137024 non-null  object
 2   endDate        137024 non-null  object
 3   sourceName     137024 non-null  object
 4   sourceVersion  137024 non-null  object
 5   startDate      137024 non-null  object
 6   type           137024 non-null  object
 7   unit           137024 non-null  object
 8   value          137024 non-null  object
dtypes: object(9)
memory usage: 9.4+ MB


In [127]:
df['endDate'] = pd.to_datetime(df.endDate)
df['startDate'] = pd.to_datetime(df.startDate)
df['creationDate'] = pd.to_datetime(df.creationDate)

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137024 entries, 0 to 137023
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype                                 
---  ------         --------------   -----                                 
 0   creationDate   137024 non-null  datetime64[ns, pytz.FixedOffset(-300)]
 1   device         137024 non-null  object                                
 2   endDate        137024 non-null  datetime64[ns, pytz.FixedOffset(-300)]
 3   sourceName     137024 non-null  object                                
 4   sourceVersion  137024 non-null  object                                
 5   startDate      137024 non-null  datetime64[ns, pytz.FixedOffset(-300)]
 6   type           137024 non-null  object                                
 7   unit           137024 non-null  object                                
 8   value          137024 non-null  object                                
dtypes: datetime64[ns, pytz.FixedOffset(-300)](3), obj

In [132]:
df = df.set_index('startDate')

In [134]:
df.value.resample('D').sum()

startDate
2016-06-03 00:00:00-05:00    700.0351199680.0305528550.0219841420.014975430...
2016-06-04 00:00:00-05:00    1580.06191962020.0776652530.0165409340.0100786...
2016-06-05 00:00:00-05:00    740.0275333440.141697360.0171312980.0363689220...
2016-06-06 00:00:00-05:00    860.02736523000.116408370.01412381110.04280636...
2016-06-07 00:00:00-05:00    1310.0417561210.0103085590.02143734070.1609291...
                                                   ...                        
2020-08-14 00:00:00-05:00    380.0161805180.0086122510.0246001760.036350269...
2020-08-15 00:00:00-05:00    750.0354989470.0227919210.0102899170.008338818...
2020-08-16 00:00:00-05:00    280.01339052710.1049312550.101831410.054978910...
2020-08-17 00:00:00-05:00    360.017485420.000969339200.009494551100.045695...
2020-08-18 00:00:00-05:00    190.00843201120.005648262890.1143821740.069531...
Freq: D, Name: value, Length: 1538, dtype: object

In [136]:
df.sourceName.value_counts()

Noah's iPhone    137018
iPhone                6
Name: sourceName, dtype: int64

In [137]:
df.sourceVersion.value_counts()

10.3.3    7841
13.3.1    7459
12.3.1    7349
10.2.1    7316
12.1.2    7301
11.3.1    6830
11.4.1    6610
10.2      5293
10.3.2    4993
10.3.1    4949
13.5.1    4902
10.0.2    4896
12.4.1    4874
12.1      4723
9.3.2     4394
12.2      4331
13.3      4138
12.1.4    3797
11.2.6    3587
11.0.3    3562
11.2.1    3533
11.2.5    3108
11.3      3100
12.4      2560
11.1.2    2356
9.3.4     2226
9.3.5     2176
10.1.1    2026
13.2.3    1633
13.4.1    1537
13.2.2    1305
9.3.1      827
9.3.3      816
12.0.1     290
13.6       256
12.0       130
Name: sourceVersion, dtype: int64

In [138]:
df.type.value_counts()

HKQuantityTypeIdentifierStepCount                 67915
HKQuantityTypeIdentifierDistanceWalkingRunning    67908
HKQuantityTypeIdentifierFlightsClimbed             1201
Name: type, dtype: int64

In [139]:
df.unit.value_counts()

count    69116
mi       67908
Name: unit, dtype: int64